In [3]:
import pandas as pd
import numpy as np 
from IPython.display import display

heart = pd.read_csv('data/heart.csv')
o2Saturation = pd.read_csv('data/o2Saturation.csv')

In [7]:
display(heart)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [49]:
from sklearn.model_selection import train_test_split
X = heart[list(heart.columns)[:-1]].values
y = heart[heart.columns[-1]].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify=y)

In [50]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(np.nan_to_num(X_train))
X_test = scaler.transform(np.nan_to_num(X_test))
X_train

array([[ 0.6417076 ,  0.65723106, -0.93751236, ..., -0.64367226,
         1.2384949 ,  1.09401427],
       [-0.0379127 ,  0.65723106,  0.98518248, ..., -0.64367226,
        -0.6951552 ,  1.09401427],
       [-0.49099291,  0.65723106,  0.98518248, ..., -0.64367226,
         0.27166985,  1.09401427],
       ...,
       [ 0.75497766,  0.65723106, -0.93751236, ..., -0.64367226,
         0.27166985, -0.52032386],
       [ 0.98151776,  0.65723106,  1.9465299 , ..., -2.26626273,
        -0.6951552 , -2.13466198],
       [ 0.6417076 , -1.52153491,  1.9465299 , ...,  0.97891822,
        -0.6951552 , -0.52032386]])

In [51]:
from xgboost  import XGBClassifier
from hyperopt import hp, fmin, tpe, Trials
from sklearn.metrics import accuracy_score

space = {
    'learning_rate': hp.loguniform('learning_rate', -5, 0),  
    'max_depth': hp.quniform('max_depth', 3, 20, 1),  
    'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),  
    'n_estimators': hp.quniform('n_estimators', 50, 200, 1),  
}

def objective(params):
    model = XGBClassifier(
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),
        min_child_weight=int(params['min_child_weight']),
        n_estimators=int(params['n_estimators']),
    )
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_val_pred)
    return 1 - accuracy

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

print("Best hyperparameters:", best)

100%|█| 100/100 [00:03<00:00, 28.18trial/s, best loss: 0.08196721
Best hyperparameters: {'learning_rate': 0.11579837357509343, 'max_depth': 16.0, 'min_child_weight': 20.0, 'n_estimators': 90.0}


In [56]:
best_xgb = XGBClassifier(
        learning_rate=0.11579837357509343,
        max_depth=int(16),
        min_child_weight=int(20),
        n_estimators=int(90)
    )
best_xgb.fit(X_train, y_train)
preds = best_xgb.predict(X_test)
acc_xgb = (preds == y_test).sum().astype(float) / len(preds)*100
print("XGBoost's prediction accuracy is: %3.2f" % (acc_xgb))

XGBoost's prediction accuracy is: 90.16


# 用不同的資料前處理對模型訓練預測是否有效果
## 在模型參數相同的情況下用對前處理方式對於訓練結果有提升

In [53]:
from sklearn.model_selection import train_test_split
X = heart[list(heart.columns)[:-1]].values
y = heart[heart.columns[-1]].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify=y)

from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
X_train_normalized = normalizer.transform(X_train)
X_test_normalized = normalizer.transform(X_test)
X_train_normalized

array([[0.22754361, 0.        , 0.        , ..., 0.0036118 , 0.        ,
        0.00722361],
       [0.22586496, 0.00337112, 0.        , ..., 0.00337112, 0.00674224,
        0.01011336],
       [0.12431093, 0.00303197, 0.00606395, ..., 0.00606395, 0.        ,
        0.00606395],
       ...,
       [0.14800684, 0.00344202, 0.        , ..., 0.00688404, 0.        ,
        0.01032606],
       [0.21889298, 0.0039088 , 0.        , ..., 0.0039088 , 0.0039088 ,
        0.0039088 ],
       [0.21545056, 0.00336641, 0.        , ..., 0.        , 0.00336641,
        0.00673283]])

In [58]:
best_xgb_normalized = XGBClassifier(
        learning_rate=0.11579837357509343,
        max_depth=int(16),
        min_child_weight=int(20),
        n_estimators=int(90)
    )
best_xgb_normalized.fit(X_train_normalized, y_train)
preds = best_xgb_normalized.predict(X_test_normalized)
acc_xgb = (preds == y_test).sum().astype(float) / len(preds)*100
print("XGBoost's prediction accuracy is: %3.2f" % (acc_xgb))

XGBoost's prediction accuracy is: 93.44
